# Day 5: If You Give A Seed A Fertilizer
This looks like a task where we can use the `chain of responsibility`.

In [ ]:
fun interface Rule {
    fun map(value: Long): Long
}

object ForwardRule : Rule {
    override fun map(value: Long) = value
}

data class MappingRule(val sourceStart: Long, val destinationStart: Long, val length: Long, private val rule: Rule) :
    Rule {
    override fun map(value: Long): Long {
        return if (value in sourceStart until sourceStart + length) {
            value + destinationStart - sourceStart
        } else
            rule.map(value)
    }
}

Rules can also be composed.

In [ ]:
fun Rule.compose(rule: Rule) = Rule { rule.map(this.map(it)) }

Some regular expressions to help getting the data

In [ ]:
val numberRegex = Regex("(\\d+)")
val mappingTitleRegex = Regex("(\\w+)-to-(\\w+) map:")

In [ ]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

val lines = Path("input.txt").readLines()

Reading the seed numbers is rather easy.

In [ ]:
val seeds = numberRegex.findAll(lines.first())
    .map { it.value.toLong() }

In [ ]:
val mappings = lines.drop(2)
    .fold(mutableMapOf<String, Pair<String, Rule>>() to "") { (m, lastKey), line ->
        when {
            // create a map entry for each section
            mappingTitleRegex.matches(line) -> {
                val (_, fst, snd) = mappingTitleRegex.find(line)!!.groupValues
                m[fst] = snd to ForwardRule
                m to fst
            }

            // setup the chain of responsibilty for each section
            line.isNotEmpty() -> {
                val (destinationStart, sourceStart, length) =
                    numberRegex.findAll(line).map { it.value.toLong() }.toList()
                m[lastKey] =
                    m[lastKey]!!.first to MappingRule(sourceStart, destinationStart, length, m[lastKey]!!.second)
                m to lastKey
            }

            // blank line, just forward what we have
            else -> m to lastKey
        }
    }.first

Compose the rules from all sections to get one big rule that we'll run on the seed numbers.

In [ ]:
var key = "seed"
var rule: Rule? = null
while (mappings.containsKey(key)) {
    val (nextKey, nextRule) = mappings[key]!!
    key = nextKey
    rule = rule?.compose(nextRule) ?: nextRule
}

## Part 1
Simply run the rules on all seed numbers and get the minimal result.

In [ ]:
seeds.map(rule::map).min()

## Part 2
I don't know if this is the most efficient way to do this, but my computer ran it in 3 to 5 minutes, so ... for me an acceptable waiting time. Using sequences here probably helps a lot with memory footprint.

I put the seed numbers into chunks and create a sequence from each. All the sequences are combined into one big sequence on which the rules are run again.

In [ ]:
seeds.chunked(2)
    .flatMap { (start, count) -> sequence { yieldAll(start until start + count) } }
    .map(rule::map)
    .min()